In [1]:
import os
import csv
import json
import time
import pandas as pd
from tqdm import tqdm

In [2]:
os.environ['HF_DATASETS_CACHE'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['CONDA_PKGS_DIRS'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['CONDA_ENVS_PATH'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['TRANSFORMERS_CACHE'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'
os.environ['HF_HOME'] = '/work/pi_dhruveshpate_umass_edu/amalgonde_umass_edu'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

/home/amalgonde_umass_edu/.conda/envs/hw1/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-16GB'

In [5]:
model_name = "microsoft/phi-2"
# model_name = "google/gemma-2b"

# config = AutoConfig.from_pretrained(model_name)
# config.max_seq_len = 4096
# config.max_answer_len= 512

tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Model max length - {tokenizer.model_max_length}")
# tokenizer.model_max_length = 4096

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             # max_length=4096,
                                             trust_remote_code=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             # load_in_8bit=True
                                            )
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Model max length - 2048


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tesla V100-PCIE-16GB


In [ ]:
# # TEST

# input_text = "Write me a poem about Machine Learning."
# input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
# # input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, padding='max_length').to("cuda")

# outputs = model.generate(**input_ids, max_new_tokens=1024)
# print(type(tokenizer.decode(outputs[0])), tokenizer.decode(outputs[0])) #.replace('<pad>', ''))

## Prep Data

### Entity Detection

In [6]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/astha/696DS-named-entity-extraction-and-linking-for-KG-construction/code/mc1/mc1_preprocess/"
file = "mc1_chunked_data.json"

with open(os.path.join(data_path, file), 'r') as f:
    data = json.load(f)

In [ ]:
type(data), data.keys()

In [ ]:
type(data['Documents']), data['Documents'].keys()

### Relation Extraction

In [7]:
entities_data = pd.read_csv("phi2_mc1_entities_processed.csv")
# Data0 = Data0.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
#                             4: 'input_tokens',  5: 'output_tokens'})
print(entities_data.shape)
entities_data.head()

(1339, 7)


,doc_key,chunk_index,entities,latency,input_tokens,output_tokens,processed_entities
0,0,0,\nKyrla Halford; University of Abila; Kronos; ...,1.578987,732,55,Kyrla Halford; University of Abila; Kronos; Pr...
1,0,1,\n\n- Sri Lanka\n- 30 June 1972\n- Udawalawe N...,3.012619,726,107,Sri Lanka; 30 June 1972; Udawalawe National Pa...
2,0,2,\n\nElodis; Kronos; 1997; grassroots effort; c...,7.108019,699,256,Elodis; Kronos; 1997; grassroots effort; conta...
3,0,3,Elodis; City Council; Kronos; Health Center; ...,7.176608,698,256,Elodis; City Council; Kronos; Health Center; C...
4,0,4,Henk Bodrogi; Kronos; Protectors of Kronos; S...,7.168755,743,256,Henk Bodrogi; Kronos; Protectors of Kronos; SM...


In [ ]:
entities_data.loc[lambda df: df.doc_key == 2]

In [8]:
data_json = {}
for idx in range(entities_data.shape[0]):
    doc_key = entities_data["doc_key"][idx]
    chunk_index = entities_data["chunk_index"][idx]
    processed_entities = entities_data["processed_entities"][idx]
    
    if doc_key not in data_json:
        data_json[doc_key] = {}
        
    data_json[doc_key][chunk_index] = processed_entities

In [ ]:
data_json.keys()

In [9]:
len(data_json[236])

3

## Prompt

### Entities

In [ ]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_entity_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    return datatmp["Text"][idx], '; '.join(list(allEntities))
    
def get_entities_prompt(text):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exout1 = get_entity_example(1, data_v2)
    ex2, exout2 = get_entity_example(2, data_v2)

    prompt=f'''Task: Please detect all the entities from the given input Text.
Entities could be people, organization, places, concepts, dates or any other proper nouns present in the text. \
Use the following examples as reference to understand the task. \
Give the output in the same format as given in the Example Entities Output, i.e., separated by a semicolon, ';'.

Example Text 1: {ex1}
Example Entities Output 1: {exout1}

Example Text 2: {ex2}
Example Entities Output 2: {exout2}

Text: {text}
Entities Output:'''
    
    return prompt

In [ ]:
sample_prompt = get_entities_prompt("text")
len(sample_prompt.split(" ")), print(sample_prompt)

### Relations

In [10]:
data_path = "/work/pi_dhruveshpate_umass_edu/project_19/ReDocREDPreprocessing/Re-DocRED/processed/"
file_v2 = "Re-DocRED_Processed_Train.csv"
    
def get_example(idx, datatmp):
    allEntities = set()
    for item in datatmp["Triplets"][idx].split("\n"):
        item = item.split(" | ")
        # print(item, len(item))
        if len(item) == 3:
            allEntities.add(item[0])
            allEntities.add(item[2])

    allEntities = list(allEntities)
    # print(allEntities)
    # return datatmp["Text"][idx], allEntities, datatmp["Triplets"][idx]
    return datatmp["Text"][idx], "; ".join(allEntities), datatmp["Triplets"][idx]
    
def get_relations_prompt(text, entities):
    
    data_v2 = pd.read_csv(os.path.join(data_path, file_v2), skiprows = range(1, 2000), nrows = 500)
    
    ex1, exent1, exout1 = get_example(1, data_v2)
    ex2, exent2, exout2 = get_example(2, data_v2)

    prompt=f'''Task Description:
The task is to extract Relations between the Entity List for given text, in the form of triplets. \
Extract triplets from the given Text based solely on the relationships present in the text. \
Ensure that entities are chosen directly from the provided Entity List to maintain accuracy. \
Avoid duplicating triplets in the output. Use the provided Example Text and Relations Output as references \
to understand how to identify meaningful relationships between entities from Entity List. \
Pay attention to all potential relations between all the entities and include them in the output.

Example Text 1: {ex1}
Entity List of Text 1: {exent1}
Relations Output of Text 1: {exout1}

Example Text 2: {ex2}
Entity List of Text 2: {exent2}
Relations Output of Text 2: {exout2}

Text: {text}
Entity List: {entities}
Relations Output:'''
    
    return prompt

In [ ]:
print(get_relations_prompt("text", "entities"))

## Predictions

In [11]:
max_tokens = 256

# lower the value, deterministic result
temperature = 0.1

# a higher value increases the chance of finding a better output
top_p = 0.9

In [12]:
output_file_name = "phi2_mc1_triplets_v3.csv"

In [13]:
from transformers import StoppingCriteria, StoppingCriteriaList

stop_list = ["Example", "Entity", "Text:", "##", "Your task", "Entities", "Output"]
stop_token_ids = [tokenizer(x,  return_tensors='pt', add_special_tokens=False)['input_ids'].to("cuda") for x in stop_list]


class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids[0]):], stop_ids[0]).all():
                return True
        return False


stopping_criteria = StoppingCriteriaList([StopOnTokens()])

### Entity Extraction

In [ ]:
output_data = []
debug = False

all_keys = list(data['Documents'].keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    for item in data['Documents'][doc_key]:
    
        #

        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # if chunk_idx == 0:
        #     continue

        # Relation prediction from GT entities
        prompt = get_entities_prompt(text)

        input_ids = tokenizer(prompt, return_tensors="pt").to("cuda") #,  max_length=4096, truncation=True

        start = time.time()
        outputs = model.generate(**input_ids,
                                 max_new_tokens=max_tokens,
                                 top_p=top_p,
                                 do_sample=True,
                                 temperature=temperature,
                                 pad_token_id=tokenizer.eos_token_id,
                                 # stopping_criteria=StoppingCriteriaList([stop_criteria]),
                                 # stopping_criteria=[Phi2StoppingCriteria()],
                                 stopping_criteria=stopping_criteria
                                 )
        time_diff = time.time() - start

        # output2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
        output2 = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

        if debug:
            print(output2)
            print(f"\nGT : {text}")

        output_data.append([doc_key, chunk_idx, output2, time_diff,
                           len(input_ids[0]), len(outputs[0][len(input_ids[0]):])])

        if i % 10 == 0 and i > 1:
            with open(output_file_name, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)

        if debug: break
        
    if debug: break
    
    with open(output_file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

### Relation Extraction

In [ ]:
data_json[0].keys()

In [14]:
output_data = []
debug = False

all_keys = list(data['Documents'].keys())

for i in tqdm(range(len(all_keys))):
    
    doc_key = all_keys[i]
    
    if int(doc_key) <= 236:
        continue
    
    for item in data['Documents'][doc_key]:
    
        #

        text = item['chunk_text']
        chunk_idx = item['chunk_index']
        
        # if chunk_idx == 0:a
        #     continue
        
        # if int(chunk_index) in data_json[int(doc_key)]:
        
        processed_entities = data_json[int(doc_key)][chunk_idx]

        # Relation prediction from GT entities
        if processed_entities:
            prompt = get_relations_prompt(text, processed_entities)

            input_ids = tokenizer(prompt, return_tensors="pt").to("cuda") #,  max_length=4096, truncation=True

            start = time.time()
            outputs = model.generate(**input_ids,
                                     max_new_tokens=max_tokens,
                                     top_p=top_p,
                                     do_sample=True,
                                     temperature=temperature,
                                     pad_token_id=tokenizer.eos_token_id,
                                     # stopping_criteria=StoppingCriteriaList([stop_criteria]),
                                     # stopping_criteria=[Phi2StoppingCriteria()],
                                     stopping_criteria=stopping_criteria
                                     )
            time_diff = time.time() - start

            # output2 = tokenizer.decode(outputs[0], skip_special_tokens=True)
            output2 = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)

            output_data.append([doc_key, chunk_idx, output2, time_diff,
                           len(input_ids[0]), len(outputs[0][len(input_ids[0]):])])

        else:
            output2 = "No entities detected"

            output_data.append([doc_key, chunk_idx, output2, time_diff,
                           0, 0])
                
#         else:
#                 output2 = "No data"

#                 output_data.append([doc_key, chunk_idx, output2, time_diff,
#                                0, 0])

        if debug:
            print(output2)
            # print(f"\nGT : {text}")

        

        if i % 10 == 0 and i > 1:
            with open(output_file_name, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerows(output_data)

        if debug: break
        
    if debug: break
    
    with open(output_file_name, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_data)

100%|██████████| 847/847 [2:38:12<00:00, 11.21s/it]  


In [ ]:
doc_key, chunk_index

In [ ]:
data_json[int(doc_key)]

In [ ]:
output_data[3]

### Process Entities

In [ ]:
Data0 = pd.read_csv("phi2_mc1_entities.csv", header=None)
Data0 = Data0.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
                            4: 'input_tokens',  5: 'output_tokens'})
Data0["processed_entities"] = ""
Data0.head()

In [ ]:
# 184, 202, 224, 261, 265, 269
Data0["entities"][34], "".split("; ")

In [ ]:
import re

for i in range(Data0.shape[0]):
    
    # if i != 261: # 1, 5, 27
    #     continue
        
    ent = Data0["entities"][i]
    ent = re.sub(r"\n\d+. ", "; ", ent)
    ent = ent.strip()
    # print(len(ent.split("\n")))
    
    if "- " in ent:
        # ent2 = re.sub("\s+", "", ent2)
        ent2 = ent.replace("\n", "")
        ent2 = ent2.replace("- ", "; ")
        ent2 = ent2.split("; ")
    
    elif len(ent.split("\n")) > 1:
        ent2 = ent.split("\n")[0].split("; ")
        
    elif ";" not in ent:
        ent2 = "".split("; ")
    
    else:
        ent2 = ent.split("; ")
        
    ent3 = []
    for item in ent2:
        item = item.strip()
        if item and item not in ent3:
            ent3.append(item)
    ent3 = "; ".join(ent3)
    
    # print(f"\n{i}")
    # print(ent)
    # print(f"\nProcessed entities - {ent3}")
    # print("*"*20)
    Data0["processed_entities"][i] = ent3

In [ ]:
Data0["processed_entities"][224]

In [ ]:
# Data0.head()
Data0.to_csv('phi2_mc1_entities_processed.csv', index=False)

In [ ]:
ent.split("\n")[0]

## Combine results

In [18]:
homeFolder = "/work/pi_dhruveshpate_umass_edu/project_19/aishwarya/696DS-named-entity-extraction-and-linking-for-KG-construction/code/phi2"

DataPath0 = os.path.join(homeFolder, "phi2_mc1_triplets_v2.csv")
DataPath1 = os.path.join(homeFolder, "phi2_mc1_triplets_v3.csv")
# DataPath2 = os.path.join(homeFolder, "phi2_relations_gt_redocred_dev_v3.csv")
# DataPath3 = os.path.join(homeFolder, "phi2_relations_gt_redocred_dev_v4.csv")

Data0 = pd.read_csv(DataPath0, header=None)
Data1 = pd.read_csv(DataPath1, header=None)
# Data2 = pd.read_csv(DataPath2, header=None)
# Data3 = pd.read_csv(DataPath3, header=None)
Data = pd.concat([Data0, Data1], axis=0)

Data = Data.rename(columns={0: 'doc_key', 1: 'chunk_index', 2: 'entities', 3: 'latency',
                            4: 'input_tokens',  5: 'output_tokens'})

Data.to_csv('phi2_mc1_triplets_combined.csv', index=False)

# print(Data.shape, len(list(set(list(Data["original_index"])))))
print(Data.shape)
Data.head()

(1339, 6)


,doc_key,chunk_index,entities,latency,input_tokens,output_tokens
0,0,0,University of Abila | sponsor | One World Res...,4.131064,1330,141
1,0,1,Sri Lanka | country | 30 June 1972\nSri Lanka...,7.259529,1359,256
2,0,2,Elodis | capital city | Kronos\nElodis | inco...,7.212500,1306,256
3,0,3,Elodis | concerned citizens | Elodis Citizens...,7.236583,1309,256
4,0,4,Kronos | community | Protectors of Kronos\nKr...,6.532060,1322,231
